In [1]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-

# imports

In [2]:
from ultralytics import YOLO
# !pip install --force-reinstall numpy scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Image prediction / classifer


In [3]:
from ultralytics import YOLO
# Load the YOLOv11 model
#model = YOLO("/kaggle/input/yolo11_trained/pytorch/default/1/yolo11m.pt")
model = YOLO("/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt")
# Predict on a sample image
#image_path =   "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000007.png" 
# image_path = "/kaggle/input/kitti-dataset/data_object_image_3/testing/image_3/000004.png"
image_path = "/kaggle/working/cropped_objects/4_Truck.png"

results = model.predict(source=image_path, conf=0.25, save=False)
# Display predictions inline (optional)
results[0].show()

FileNotFoundError: /kaggle/working/cropped_objects/4_Truck.png does not exist

# Grad CAM current


In [ ]:
# i = 7
import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Load YOLOv11 model
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Load and preprocess image
#image_path = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000007.png"
image_path = "/kaggle/input/ig-test-on-1st-car-in-oooo7/save as IG test.png"
original_image = Image.open(image_path).convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize(( 320,736)),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

# Get class names and find 'Car' index
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
#tram_class_index = [k for k, v in class_names.items() if v.lower() == "tram"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]
#tram_class_index = tram_class_index[0]

# Hook Grad-CAM
activations = None
gradients = None

def forward_hook(module, input, output):
    global activations
    activations = output

def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

# Register hooks
target_layer = model.model.model[16]  # Should be a deep conv layer
target_layer.register_forward_hook(forward_hook)
target_layer.register_full_backward_hook(backward_hook)

# Forward pass through raw model
output = model.model(input_tensor)[0]  # shape: (1, N, 5 + num_classes)
output = output[0]  # Remove batch dim

# Compute car class scores
obj_conf = output[:, 4]
class_probs = output[:, 5:]
car_scores = obj_conf * class_probs[:, car_class_index]

# Select top car
if car_scores.max() == 0:
    raise ValueError("No car detected in raw output.")
    
top_idx = torch.argmax(car_scores)
score = car_scores[top_idx]
# Get top-k car detections
# top_k = 5  # You can tune this number
# topk_scores, topk_indices = torch.topk(car_scores, k=top_k)

# # Filter zero-score predictions
# non_zero = topk_scores > 0
# if non_zero.sum() == 0:
#     raise ValueError("No high-scoring car predictions.")
# filtered_scores = topk_scores[non_zero]

# # Take mean score for Grad-CAM
# score = filtered_scores.mean()



# Backward pass for Grad-CAM
model.model.zero_grad()
score.backward(retain_graph=True)

# Grad-CAM computation
weights = gradients.mean(dim=(2, 3), keepdim=True)
cam = (weights * activations).sum(dim=1, keepdim=True)
cam = F.relu(cam)

# Normalize CAM using Clipping Percentiles
cam = cam.squeeze().detach().cpu().numpy()
cam = cv2.resize(cam, (640, 640))

# Compute 1st and 99th percentiles
p1, p99 = np.percentile(cam, 1), np.percentile(cam, 99)

# Clip and rescale to [0, 1]
cam = np.clip(cam, p1, p99)
cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-6)

# Create heatmap overlay
original_np = np.array(original_image.resize((640, 640))) / 255.0
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
heatmap = np.float32(heatmap) / 255
overlay = heatmap + original_np
overlay = overlay / np.max(overlay)

# Display
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title("Original")
plt.imshow(original_np)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Grad-CAM Heatmap")
plt.imshow(cam, cmap="jet")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Overlay")
plt.imshow(overlay)
plt.axis("off")

plt.tight_layout()
plt.show()

# Show detection result
results[0].show()


### gradCAM with prevuilt libaries - the prebuilt library is not supported in kaggle

In [ ]:
!pip install -q pytorch-grad-cam

In [ ]:
# Install if not done
# !pip install -q pytorch-grad-cam

import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# Load YOLOv11 model
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Load and preprocess image
image_path = "/kaggle/input/red-car-on-road/road-landscape-red-car-beautiful-260nw-2377086843.png"
original_image = Image.open(image_path).convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)

# Find 'car' class index
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]

# Forward pass through raw model
output = model.model(input_tensor)[0]  # shape: (1, N, 5 + num_classes)
output = output[0]  # Remove batch dim

# Compute car class scores
obj_conf = output[:, 4]
class_probs = output[:, 5:]
car_scores = obj_conf * class_probs[:, car_class_index]

# Select top car
if car_scores.max() == 0:
    raise ValueError("No car detected in raw output.")
top_idx = torch.argmax(car_scores)

# -------------------------------
# Grad-CAM Part (library version)
# -------------------------------

from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# You must define a custom target
class DetectionTarget:
    def __init__(self, index):
        self.index = index

    def __call__(self, model_output):
        """
        model_output shape: [num_preds, 5 + num_classes]
        We are focusing on objectness * class prob of 'car' at specific index
        """
        obj_conf = model_output[:, 4]
        class_probs = model_output[:, 5:]
        car_score = obj_conf * class_probs[:, car_class_index]
        return car_score[self.index]

# Select a good layer
target_layer = model.model.model[16]  # Same deep conv layer

cam = GradCAM(model=model.model, target_layers=[target_layer], use_cuda=torch.cuda.is_available())

targets = [DetectionTarget(index=top_idx)]

grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]  # [0] since batch size = 1

# Prepare visualization
original_np = np.array(original_image.resize((640, 640))) / 255.0
visualization = show_cam_on_image(original_np, grayscale_cam, use_rgb=True)

# Plot
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title("Original")
plt.imshow(original_np)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Grad-CAM Heatmap")
plt.imshow(grayscale_cam, cmap="jet")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Overlay")
plt.imshow(visualization)
plt.axis("off")

plt.tight_layout()
plt.show()

# Optionally show detection result
results = model.predict(image_path)
results[0].show()


## Grad CAm error solving 

In [ ]:


# Load YOLOv11 model
#model_path = "/kaggle/input/trained_best-model/pytorch/default/1/best.pt"
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Load and preprocess image
image_path = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000007.png"
original_image = Image.open(image_path).convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize((384, 1248)),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

# Get class names
class_names = model.names  # dict {id: name}

# Hook Grad-CAM
activations = None
gradients = None

def forward_hook(module, input, output):
    global activations
    activations = output

def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

# Register hooks
target_layer = model.model.model[16]  # Should be a deep conv layer
target_layer.register_forward_hook(forward_hook)
target_layer.register_full_backward_hook(backward_hook)

# Forward pass through YOLO model
#preds = model.predict(original_image, imgsz=(384, 1248), conf=0.3, device=device)[0]
preds = model.predict(input_tensor, imgsz=(384, 1248), conf=0.3, device=device)[0]

pred_classes = preds.boxes.cls.int().tolist()

# Get unique detected classes
detected_class_indices = list(set(pred_classes))
print(f"✅ Detected classes: {[class_names[idx] for idx in detected_class_indices]}")

# Forward pass through raw model for Grad-CAM
output = model.model(input_tensor)[0][0]  # Remove batch dim
obj_conf = output[:, 4]
class_probs = output[:, 5:]

# Prepare original image as numpy
#original_np = np.array(original_image.resize()) / 255.0
original_np = np.array(original_image.resize((1248, 384))) / 255.0

cams_per_class = {}

# Grad-CAM for each detected class
for class_idx in detected_class_indices:
    class_name = class_names[class_idx]
    print(f"🔵 Creating Grad-CAM for {class_name}")
    
    class_scores = obj_conf * class_probs[:, class_idx]
    if class_scores.max() == 0:
        print(f"⚠️ No {class_name} detected in raw output. Skipping.")
        continue
    
    top_idx = torch.argmax(class_scores)
    score = class_scores[top_idx]
    
    model.model.zero_grad()
    score.backward(retain_graph=True)
    
    weights = gradients.mean(dim=(2, 3), keepdim=True)
    cam = (weights * activations).sum(dim=1, keepdim=True)
    cam = F.relu(cam).squeeze().detach().cpu().numpy()
    cam = cv2.resize(cam, (1248, 384))
    
    p1, p99 = np.percentile(cam, 1), np.percentile(cam, 99)
    cam = np.clip(cam, p1, p99)
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-6)
    
    cams_per_class[class_name] = cam

# ===================== 🔥 Updated Visualization 🔥 =====================

num_classes_detected = len(cams_per_class)
total_images = num_classes_detected + 1  # +1 for the original image

#plt.figure(figsize=(5 * total_images, 5))  # Wide figure for 1 row
fig_width = min(5 * total_images, 30)  # max width 30 inches
plt.figure(figsize=(fig_width, 5))


# Original Image
plt.subplot(1, total_images, 1)
plt.title("Original Image", fontsize=14, color='black')
plt.imshow(original_np)
plt.axis("off")

# Individual CAMs
for idx, (class_name, cam) in enumerate(cams_per_class.items(), start=2):
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    overlay = heatmap + original_np
    overlay = overlay / np.max(overlay)
    
    plt.subplot(1, total_images, idx)
    plt.title(f"{class_name} - Grad-CAM", fontsize=14, color='black')
    plt.imshow(overlay)
    plt.axis("off")

plt.tight_layout()
plt.show()



## IG 


In [ ]:
!pip install captum


In [ ]:
# --- Imports ---
from captum.attr import IntegratedGradients

# --- Define Baseline ---
# Same size as input, black image
baseline = torch.zeros_like(input_tensor).to(device)

# --- Define prediction function ---
def predict(inputs):
    outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
    outputs = outputs[0]  # Remove batch dim
    
    obj_conf = outputs[:, 4]
    class_probs = outputs[:, 5:]
    car_scores = obj_conf * class_probs[:, car_class_index]
    
    top_idx = torch.argmax(car_scores)
    score = car_scores[top_idx]
    
    return score

# --- Create Integrated Gradients object ---
ig = IntegratedGradients(predict)

# --- Compute attributions ---
attributions, delta = ig.attribute(
    inputs=input_tensor,
    baselines=baseline,
    target=None,   # because we already handle scoring inside predict
    return_convergence_delta=True,
    n_steps=5,    # More steps = smoother but slower
)

# --- Process attributions ---
# Take absolute value and sum across channels
attr = attributions.squeeze().detach().cpu().numpy()
attr = np.sum(np.abs(attr), axis=0)

# Normalize for visualization
attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)

# --- Visualization ---
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.title("Original")
plt.imshow(original_np)
plt.axis("off")

plt.subplot(1,3,2)
plt.title("Integrated Gradients")
plt.imshow(attr, cmap="viridis")
plt.axis("off")

plt.subplot(1,3,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# --- Imports ---
import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from captum.attr import IntegratedGradients

# --- Optional: Speed up conv layers ---
torch.backends.cudnn.benchmark = True

# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Load and preprocess image ---
image_path = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000007.png"
original_image = Image.open(image_path).convert("RGB")

# Resize to 384 x 640
resize_size = (640, 384)
preprocess = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

# --- Baseline (black image) ---
baseline = torch.zeros_like(input_tensor).to(device)

# --- Class Names ---
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]

# --- Define prediction function ---
# def predict(inputs):
#     outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
#     outputs = outputs[0]  # remove batch dim
#     obj_conf = outputs[:, 4]
#     class_probs = outputs[:, 5:]
#     car_scores = obj_conf * class_probs[:, car_class_index]
#     top_idx = torch.argmax(car_scores)
#     score = car_scores[top_idx]
#     return score
def predict(inputs):
    outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
    outputs = outputs[0]  # remove batch dim
    obj_conf = outputs[:, 4]
    class_probs = outputs[:, 5:]
    car_scores = obj_conf * class_probs[:, car_class_index]
    top_idx = torch.argmax(car_scores)
    score = car_scores[top_idx]
    score = score.unsqueeze(0)  # <--- fix for 0D tensors
    return score


# --- Integrated Gradients object ---
ig = IntegratedGradients(predict)

# --- Compute attributions ---
attributions, delta = ig.attribute(
    inputs=input_tensor,
    baselines=baseline,
    target=None,
    return_convergence_delta=True,
    n_steps=20,               # fewer steps
    internal_batch_size=4     # mini-batches
)

# --- Process attributions ---
attr = attributions.squeeze().detach().cpu().numpy()
attr = np.sum(np.abs(attr), axis=0)  # sum over channels
attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)  # normalize

# --- Prepare original resized image for overlay ---
original_np = np.array(original_image.resize(resize_size)) / 255.0

# # Before combining
# original_np = cv2.resize(original_np, (heatmap_ig.shape[1], heatmap_ig.shape[0]))


# --- Visualization ---
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.title("Original")
plt.imshow(original_np)
plt.axis("off")

plt.subplot(1,3,2)
plt.title("Integrated Gradients")
plt.imshow(attr, cmap="viridis")
plt.axis("off")

plt.subplot(1,3,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()

# IG Implemented

In [ ]:
import cv2
import matplotlib.pyplot as plt
# Load the image (in color)
image = cv2.imread("/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000007.png")  # This loads as BGR by default

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Save or display the grayscale image (optional)
cv2.imwrite("path_to_save_gray_image.jpg", gray_image)

# To display (for testing):
cv2.imshow("Grayscale Image", gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# --- Imports ---
import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from captum.attr import IntegratedGradients
import torchvision.transforms.functional as TF

# --- Speed up ---
torch.backends.cudnn.benchmark = True

# --- Load Model ---
model_path = "/kaggle/input/yolov11m/pytorch/default/1/yolo11m.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Load and preprocess image ---
image_path = "/kaggle/input/ig-test-on-1st-car-in-oooo7/save as IG test.png"
original_image = Image.open(image_path).convert("RGB")

resize_size = (640,640)
preprocess = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

# --- Blurred baseline ---
def blur_baseline(image_tensor, kernel_size=51):
    blurred = TF.gaussian_blur(image_tensor.cpu().squeeze(0), kernel_size=kernel_size)
    blurred = blurred.unsqueeze(0).to(image_tensor.device)
    return blurred

baseline = blur_baseline(input_tensor, kernel_size=51)

# --- Class Names ---
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]

# --- Define prediction function ---
def predict(inputs):
    outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
    outputs = outputs[0]
    obj_conf = outputs[:, 4]
    class_probs = outputs[:, 5:]
    car_scores = obj_conf * class_probs[:, car_class_index]
    top_idx = torch.argmax(car_scores)
    score = car_scores[top_idx]
    return score.unsqueeze(0)

# --- Integrated Gradients ---
ig = IntegratedGradients(predict)

# --- Compute attributions ---
attributions, delta = ig.attribute(
    inputs=input_tensor,
    baselines=baseline,
    target=None,
    return_convergence_delta=True,
    n_steps=300,               # more steps = better quality
    internal_batch_size=6
)

# --- Process attributions ---
attr = attributions.squeeze().detach().cpu().numpy()
attr = np.sum(np.abs(attr), axis=0)  # sum over channels
attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)  # normalize

# --- Prepare image for visualization ---
original_np = np.array(original_image.resize(resize_size)) / 255.0

# --- Draw bounding boxes ---
results = model.predict(original_image, imgsz=resize_size, conf=0.3)
boxes = results[0].boxes.xyxy.cpu().numpy()

for box in boxes:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(original_np, (x1, y1), (x2, y2), color=(1,0,0), thickness=2)

# --- Visualization ---
plt.figure(figsize=(15,5))

# Plot Original Image with Bounding Boxes
plt.subplot(1,3,1)
plt.title("Original + Detections")
plt.imshow(original_np)
plt.axis("off")

# Plot Integrated Gradients Attribution Map
plt.subplot(1,3,2)
plt.title("Integrated Gradients")
plt.imshow(attr, cmap="")
plt.axis("off")

# Plot Overlay (heatmap + original)
plt.subplot(1,3,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_JET)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# --- Run detection on the resized input image ---
detections = model.predict(original_image.resize(resize_size), conf=0.25)[0]  # confidence threshold 25%

# --- Extract 'car' detections ---
car_boxes = []
for box in detections.boxes:
    if int(box.cls.item()) == car_class_index:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        car_boxes.append((int(x1), int(y1), int(x2), int(y2)))

# --- Draw bounding boxes on overlay ---
overlay_with_boxes = (overlay_ig * 255).astype(np.uint8).copy()
overlay_with_boxes = cv2.cvtColor(overlay_with_boxes, cv2.COLOR_RGB2BGR)  # OpenCV expects BGR

for (x1, y1, x2, y2) in car_boxes:
    cv2.rectangle(overlay_with_boxes, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)  # green boxes

overlay_with_boxes = cv2.cvtColor(overlay_with_boxes, cv2.COLOR_BGR2RGB)  # back to RGB for matplotlib

# --- Visualization ---
plt.figure(figsize=(18,5))

# Original
plt.subplot(1,4,1)
plt.title("Original")
plt.imshow(original_np)
plt.axis("off")

# Integrated Gradients Heatmap
plt.subplot(1,4,2)
plt.title("IG Heatmap")
plt.imshow(attr, cmap="jet")
plt.axis("off")

# Overlay
plt.subplot(1,4,3)
plt.title("Overlay (no boxes)")
plt.imshow(overlay_ig)
plt.axis("off")

# Overlay + Bounding Boxes
plt.subplot(1,4,4)
plt.title("Overlay + Car Bounding Boxes")
plt.imshow(overlay_with_boxes)
plt.axis("off")

plt.tight_layout()
plt.show()


# BEST IG TILL NOW

In [ ]:
# --- Imports ---
import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from captum.attr import IntegratedGradients
import torchvision.transforms.functional as TF

# --- Speed up ---
torch.backends.cudnn.benchmark = True

# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Load and preprocess image ---
image_path = "/kaggle/input/ig-test-on-1st-car-in-oooo7/save as IG test.png"
original_image = Image.open(image_path).convert("RGB")

resize_size = (640,640)
preprocess = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

# --- Blurred baseline ---
def blur_baseline(image_tensor, kernel_size=51):
    blurred = TF.gaussian_blur(image_tensor.cpu().squeeze(0), kernel_size=kernel_size)
    blurred = blurred.unsqueeze(0).to(image_tensor.device)
    return blurred

baseline = blur_baseline(input_tensor, kernel_size=51)

# --- Class Names ---
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]

# --- Define prediction function ---
def predict(inputs):
    outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
    outputs = outputs[0]
    obj_conf = outputs[:, 4]
    class_probs = outputs[:, 5:]
    car_scores = obj_conf * class_probs[:, car_class_index]
    top_idx = torch.argmax(car_scores)
    score = car_scores[top_idx]
    return score.unsqueeze(0)

# --- Integrated Gradients ---
ig = IntegratedGradients(predict)

# --- Compute attributions ---
attributions, delta = ig.attribute(
    inputs=input_tensor,
    baselines=baseline,
    target=None,
    return_convergence_delta=True,
    n_steps=300,               # more steps = better quality
    internal_batch_size=6
)

# --- Process attributions ---
attr = attributions.squeeze().detach().cpu().numpy()
attr = np.sum(np.abs(attr), axis=0)  # sum over channels
attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)  # normalize

# --- Prepare image for visualization ---
original_np = np.array(original_image.resize(resize_size)) / 255.0

# --- Draw bounding boxes ---
results = model.predict(original_image, imgsz=resize_size, conf=0.3)
boxes = results[0].boxes.xyxy.cpu().numpy()

for box in boxes:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(original_np, (x1, y1), (x2, y2), color=(1,0,0), thickness=2)

# --- Visualization ---
plt.figure(figsize=(15,5))

# Plot Original Image with Bounding Boxes
plt.subplot(1,3,1)
plt.title("Original + Detections")
plt.imshow(original_np)
plt.axis("off")

# Plot Integrated Gradients Attribution Map
plt.subplot(1,3,2)
plt.title("Integrated Gradients")
plt.imshow(attr, cmap="jet")
plt.axis("off")

# Plot Overlay (heatmap + original)
plt.subplot(1,3,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_JET)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()



In [ ]:
# --- Visualization with multiple colormap options ---
plt.figure(figsize=(15,5))

# Plot Original Image with Bounding Boxes
plt.subplot(1,3,1)
plt.title("Original + Detections")
plt.imshow(original_np)
plt.axis("off")

# Plot Integrated Gradients Attribution Map
plt.subplot(1,3,2)
plt.title("Integrated Gradients")

# Here we use the viridis colormap
plt.imshow(attr, cmap='inferno')  # Change 'viridis' to other colormaps like 'plasma', 'inferno', etc.
plt.axis("off")

# Plot Overlay (heatmap blended with original)
plt.subplot(1,3,3)
plt.title("Overlay")

# Make sure attr is clipped to 0-1
attr = np.clip(attr, 0, 1)

# Resize heatmap to match the image size (safety)
heatmap_attr = cv2.resize(attr, resize_size)

# Use colormap, e.g., Viridis (or try 'inferno', 'plasma', etc.)
heatmap_ig = cv2.applyColorMap(np.uint8(255 * heatmap_attr), cv2.COLORMAP_VIRIDIS)  # Change here for 'viridis'
heatmap_ig = cv2.cvtColor(heatmap_ig, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR, matplotlib expects RGB
heatmap_ig = np.float32(heatmap_ig) / 255

# Blend heatmap and original image with adjustable transparency (alpha)
alpha = 0.5  # You can change alpha here to experiment with transparency
overlay_ig = (1 - alpha) * original_np + alpha * heatmap_ig
overlay_ig = np.clip(overlay_ig, 0, 1)

# Show the final overlay
plt.imshow(overlay_ig)
plt.axis("off")

# Final display
plt.tight_layout()
plt.show()


In [ ]:
import torch

torch.cuda.empty_cache()


In [ ]:
# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

In [ ]:
print(input_tensor.device)
print(baseline.device)
print(next(model.model.parameters()).device)


In [ ]:
# Check the format of the detections
outputs = model(input_tensor)[0]  # Outputs from the YOLO model
print(outputs)


## IG on or Inside teh bounding boxes

In [ ]:
import torch
from ultralytics import YOLO
from PIL import Image
import numpy as np
from torchvision import transforms

# Load YOLO model (replace with your actual path)
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)

# Ensure the model is in evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Load Image (replace with your actual image path)
image_path = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000007.png"
original_image = Image.open(image_path).convert("RGB")

# Preprocess image to match YOLO input
resize_size = (384, 1248)
preprocess = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()


In [ ]:
# Run Object Detection on the image
outputs = model(input_tensor)[0]  # Model output

# The 'outputs' is a 'Results' object. We'll extract the detections
detections = outputs.boxes.xyxy.cpu().numpy()  # Bounding boxes (x1, y1, x2, y2)
confidences = outputs.boxes.conf.cpu().numpy()  # Confidence scores for the boxes
class_probs = outputs.boxes.cls.cpu().numpy()  # Class probabilities for the boxes

# Class names (this is a dictionary)
class_names = model.names

# Print all class names for debugging
print("Class Names:", class_names)

# Get index of "Car" class (case-insensitive search)
car_class_index = next((k for k, v in class_names.items() if v.lower() == "car"), None)

# Check if we found the "Car" class and print the index
if car_class_index is not None:
    print("Found Car class at index:", car_class_index)
else:
    print("Car class not found. Check class names:", class_names)


In [ ]:
# --- Extract Car Detections (Filter with Confidence Threshold) ---
car_detections = []

# Loop through all detections
for i, cls_prob in enumerate(class_probs):
    if cls_prob == car_class_index:  # Check if it's a 'car'
        confidence = confidences[i]  # Confidence score for the detection
        if confidence > 0.5:  # Filter out low confidence detections
            car_detections.append(detections[i])  # Add detection to car_detections

# Check how many detections we have for 'car'
print(f"Found {len(car_detections)} car detections above 0.5 confidence.")

# Let's view one of the detections for debugging purposes
print("One of the detected bounding boxes:", car_detections[0])


# Image Croppings

In [ ]:
import os
import shutil

# --- DELETE old cropped_outputs folder if it exists ---
save_cropped_dir = "cropped_outputs"

if os.path.exists(save_cropped_dir):
    shutil.rmtree(save_cropped_dir)
os.makedirs(save_cropped_dir, exist_ok=True)


In [ ]:
import os
from PIL import Image
import torch
from ultralytics import YOLO

# --- Paths ---
image_path = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000010.png"   # your input image
save_crop_folder = "cropped_objects"
os.makedirs(save_crop_folder, exist_ok=True)

# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Load image ---
img = Image.open(image_path).convert("RGB")
img_np = np.array(img)

# --- Run Detection ---
results = model.predict(img, imgsz=(640, 640), conf=0.3)
boxes = results[0].boxes.xyxy.cpu().numpy()
pred_classes = results[0].boxes.cls.cpu().numpy()

class_names = model.names

# --- Crop and Save Each Object ---
for idx, (box, cls_id) in enumerate(zip(boxes, pred_classes)):
    x1, y1, x2, y2 = map(int, box)
    cropped_obj = img.crop((x1, y1, x2, y2))

    class_name = class_names[int(cls_id)]
    save_path = os.path.join(save_crop_folder, f"{idx}_{class_name}.png")
    cropped_obj.save(save_path)
    print(f"Saved: {save_path}")

print(f"✅ Done! Cropped {len(boxes)} objects and saved them in '{save_crop_folder}/'")


## IG on cropped detection images

### here th IG is currentluy being calculated on the complete image tensor and not the cropped counded boxes

In [ ]:
import torch
from ultralytics import YOLO
from PIL import Image
import numpy as np
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt
from captum.attr import IntegratedGradients

# Load YOLO model
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)

# Ensure the model is in evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# Load Image
image_path = "/kaggle/working/misclassified/000009.png"
original_image = Image.open(image_path).convert("RGB")

# Preprocess image to match YOLO input
resize_size = (384, 1248)
preprocess = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.ToTensor()
])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

# Run Object Detection on the
outputs = model(input_tensor)[0]  # Model output

# The 'outputs' is a 'Results' object. We'll extract the detections
detections = outputs.boxes.xyxy.cpu().numpy()  # Bounding boxes (x1, y1, x2, y2)
confidences = outputs.boxes.conf.cpu().numpy()  # Confidence scores for the boxes
class_probs = outputs.boxes.cls.cpu().numpy()  # Class probabilities for the boxes

# Class names (this is a dictionary)
class_names = model.names

# Get index of "Car" class
car_class_index = next((k for k, v in class_names.items() if v.lower() == "Pedestrian"), None)

# --- Extract Car Detections (Filter with Confidence Threshold) ---
car_detections = []

# Loop through all detections
for i, cls_prob in enumerate(class_probs):
    if cls_prob == car_class_index:  # Check if it's a 'car'
        confidence = confidences[i]  # Confidence score for the detection
        if confidence > 0.5:  # Filter out low confidence detections
            car_detections.append(detections[i])  # Add detection to car_detections

# Print how many detections we have for 'car'
print(f"Found {len(car_detections)} car detections above 0.5 confidence.")

# Let's view one of the detections for debugging purposes
print("One of the detected bounding boxes:", car_detections[0])

# Define your Integrated Gradients object
ig = IntegratedGradients(predict)

# Visualize attributions inside the bounding boxes
for det in car_detections:
    x1, y1, x2, y2 = map(int, det[:4])  # Bounding box coordinates
    bbox_image = np.array(original_image)[y1:y2, x1:x2, :]  # Crop the image inside the bounding box

    # Convert to tensor and preprocess for IG
    bbox_tensor = preprocess(Image.fromarray(bbox_image)).unsqueeze(0).to(device)

    # Create a baseline (black image) for IG
    baseline = torch.zeros_like(bbox_tensor).to(device)

    # Compute IG for the Bounding Box Region
    attributions, delta = ig.attribute(
        inputs=bbox_tensor,
        baselines=baseline,
        target=None,
        return_convergence_delta=True,
        n_steps=50,  # More steps for better resolution
        internal_batch_size=4  # Suitable batch size
    )

    # Process the attributions to visualize them
    attr = attributions.squeeze().detach().cpu().numpy()
    attr = np.sum(np.abs(attr), axis=0)  # Sum over the channels (RGB)
    attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)  # Normalize the attribution map

    # --- Prepare for Visualization ---
    # Apply a colormap to the attribution map
    heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
    heatmap_ig = np.float32(heatmap_ig) / 255  # Normalize to 0-1 range

    # Resize the heatmap to match the size of the bounding box
    heatmap_ig_resized = cv2.resize(heatmap_ig, (bbox_image.shape[1], bbox_image.shape[0]))

    # Overlay the resized heatmap on the original cropped image
    overlay_ig = heatmap_ig_resized + bbox_image / 255.0
    overlay_ig = overlay_ig / np.max(overlay_ig)  # Normalize to [0, 1]

    # --- Plot Results ---
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 3, 1)
    plt.title("Original ROI (Bounding Box)")
    plt.imshow(bbox_image)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Attribution Map")
    plt.imshow(attr, cmap="viridis")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Overlay")
    plt.imshow(overlay_ig)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
# --- Imports ---
import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from captum.attr import IntegratedGradients
import torchvision.transforms.functional as TF

# --- Speed up ---
torch.backends.cudnn.benchmark = True

# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Resize with padding function (Letterbox style) ---
def resize_with_padding(image, target_size=(640, 640)):
    old_w, old_h = image.size
    target_w, target_h = target_size
    scale = min(target_w / old_w, target_h / old_h)
    new_w, new_h = int(old_w * scale), int(old_h * scale)
    resized_image = image.resize((new_w, new_h), Image.BILINEAR)
    new_image = Image.new("RGB", target_size, (0, 0, 0))  # black padding
    paste_x = (target_w - new_w) // 2
    paste_y = (target_h - new_h) // 2
    new_image.paste(resized_image, (paste_x, paste_y))
    return new_image, scale, paste_x, paste_y

# --- Blurred baseline ---
def blur_baseline(image_tensor, kernel_size=51):
    blurred = TF.gaussian_blur(image_tensor.cpu().squeeze(0), kernel_size=kernel_size)
    blurred = blurred.unsqueeze(0).to(image_tensor.device)
    return blurred

# --- Load and preprocess image ---
image_path = "/kaggle/input/car-imge/car_lifestyle-02.jpg"
original_image = Image.open(image_path).convert("RGB")

resize_size = (640, 640)
resized_padded_image, scale, pad_x, pad_y = resize_with_padding(original_image, target_size=resize_size)

preprocess = transforms.Compose([
    transforms.ToTensor()
])
input_tensor = preprocess(resized_padded_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

baseline = blur_baseline(input_tensor, kernel_size=51)

# --- Class Names ---
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]

# --- Define prediction function ---
def predict(inputs):
    outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
    outputs = outputs[0]
    obj_conf = outputs[:, 4]
    class_probs = outputs[:, 5:]
    car_scores = obj_conf * class_probs[:, car_class_index]
    top_idx = torch.argmax(car_scores)
    score = car_scores[top_idx]
    return score.unsqueeze(0)

# --- Integrated Gradients ---
ig = IntegratedGradients(predict)

# --- Compute attributions ---
attributions, delta = ig.attribute(
    inputs=input_tensor,
    baselines=baseline,
    target=None,
    return_convergence_delta=True,
    n_steps=300,
    internal_batch_size=6
)

# --- Process attributions ---
attr = attributions.squeeze().detach().cpu().numpy()
attr = np.sum(np.abs(attr), axis=0)  # sum over channels
attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)  # normalize

# --- Run Detection ---
results = model.predict(resized_padded_image, imgsz=resize_size, conf=0.3)
boxes = results[0].boxes.xyxy.cpu().numpy()

# --- Prepare image for visualization ---
original_np = np.array(resized_padded_image) / 255.0

# --- Draw bounding boxes ---
for box in boxes:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(original_np, (x1, y1), (x2, y2), color=(1,0,0), thickness=2)

# --- Visualization ---
plt.figure(figsize=(18,6))

# Original + Detections
plt.subplot(1,3,1)
plt.title("Original + Detections")
plt.imshow(original_np)
plt.axis("off")

# Integrated Gradients Heatmap
plt.subplot(1,3,2)
plt.title("Integrated Gradients (viridis)")
plt.imshow(attr, cmap="viridis")
plt.axis("off")

# Overlay (Heatmap + Original)
plt.subplot(1,3,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()


# BEST IG + work on BOUNDING BOX

In [ ]:
# --- Imports ---
import torch
import torch.nn.functional as F
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from captum.attr import IntegratedGradients
import torchvision.transforms.functional as TF

# --- Speed up ---
torch.backends.cudnn.benchmark = True

# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Resize with padding function ---
def resize_with_padding(image, target_size=(640, 640)):
    old_w, old_h = image.size
    target_w, target_h = target_size
    scale = min(target_w / old_w, target_h / old_h)
    new_w, new_h = int(old_w * scale), int(old_h * scale)
    resized_image = image.resize((new_w, new_h), Image.BILINEAR)
    new_image = Image.new("RGB", target_size, (0, 0, 0))  # black padding
    paste_x = (target_w - new_w) // 2
    paste_y = (target_h - new_h) // 2
    new_image.paste(resized_image, (paste_x, paste_y))
    return new_image, scale, paste_x, paste_y

# --- Blurred baseline ---
def blur_baseline(image_tensor, kernel_size=51):
    blurred = TF.gaussian_blur(image_tensor.cpu().squeeze(0), kernel_size=kernel_size)
    blurred = blurred.unsqueeze(0).to(image_tensor.device)
    return blurred

# --- Load and preprocess image ---
# image_path = "/kaggle/input/ig-test-on-1st-car-in-oooo7/save as IG test.png"
image_path ="/kaggle/working/cropped_objects/5_Van.png"
# image_path = "/kaggle/working/misclassified/missed_car_000027.png"
original_image = Image.open(image_path).convert("RGB")

# --- Change these target sizes here ---
# target_size = (736, 320)    # <- first try
target_size = (1248,384)    # <- then change to this

resized_padded_image, scale, pad_x, pad_y = resize_with_padding(original_image, target_size=target_size)

preprocess = transforms.Compose([
    transforms.ToTensor()
])
input_tensor = preprocess(resized_padded_image).unsqueeze(0).to(device)
input_tensor.requires_grad_()

baseline = blur_baseline(input_tensor, kernel_size=51)

# --- Class Names ---
class_names = model.names
car_class_index = [k for k, v in class_names.items() if v.lower() == "car"]
if not car_class_index:
    raise ValueError("Car class not found.")
car_class_index = car_class_index[0]

# --- Define prediction function ---
def predict(inputs):
    outputs = model.model(inputs)[0]  # shape: (1, N, 5+num_classes)
    outputs = outputs[0]
    obj_conf = outputs[:, 4]
    class_probs = outputs[:, 5:]
    car_scores = obj_conf * class_probs[:, car_class_index]
    top_idx = torch.argmax(car_scores)
    score = car_scores[top_idx]
    return score.unsqueeze(0)

# --- Integrated Gradients ---
ig = IntegratedGradients(predict)

# --- Compute attributions ---
attributions, delta = ig.attribute(
    inputs=input_tensor,
    baselines=baseline,
    target=None,
    return_convergence_delta=True,
    n_steps=300,
    internal_batch_size=6
)

# --- Process attributions ---
attr = attributions.squeeze().detach().cpu().numpy()
attr = np.sum(np.abs(attr), axis=0)
attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-6)

# --- Run Detection ---
results = model.predict(resized_padded_image, imgsz=target_size, conf=0.3)
boxes = results[0].boxes.xyxy.cpu().numpy()

# --- Prepare image for visualization ---
original_np = np.array(resized_padded_image) / 255.0

# --- Draw bounding boxes ---
for box in boxes:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(original_np, (x1, y1), (x2, y2), color=(1,0,0), thickness=2)

# --- Visualization ---
plt.figure(figsize=(18,6))

# Original + Detections
plt.subplot(1,3,1)
plt.title(f"Original + Detections ({target_size[0]}x{target_size[1]})")
plt.imshow(original_np)
plt.axis("off")

# Integrated Gradients Heatmap
plt.subplot(1,3,2)
plt.title("Integrated Gradients (viridis)")
plt.imshow(attr, cmap="viridis")
plt.axis("off")

# Overlay (Heatmap + Original)
plt.subplot(1,3,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12,18))  # (width, height) -> make it taller now

# 1st Plot: Original + Detections
plt.subplot(3,1,1)
plt.title(f"Original + Detections ({target_size[0]}x{target_size[1]})")
plt.imshow(original_np)
plt.axis("off")

# 2nd Plot: Integrated Gradients
plt.subplot(3,1,2)
plt.title("Integrated Gradients (viridis)")
plt.imshow(attr, cmap="viridis")
plt.axis("off")

# 3rd Plot: Overlay
plt.subplot(3,1,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")

plt.tight_layout()
plt.show()


## Saving the above work

In [ ]:
import os

# --- Visualization and Saving ---
save_dir = "saved_outputs"
os.makedirs(save_dir, exist_ok=True)  # Create folder if not exists

plt.figure(figsize=(8, 24))  # Taller figure for vertical layout

# Original + Detections
plt.subplot(3,1,1)
plt.title(f"Original + Detections ({target_size[0]}x{target_size[1]})")
plt.imshow(original_np)
plt.axis("off")
plt.savefig(os.path.join(save_dir, "Original_with_Detections.png"), bbox_inches='tight', pad_inches=0.1)

# Integrated Gradients Heatmap
plt.subplot(3,1,2)
plt.title("Integrated Gradients (viridis)")
plt.imshow(attr, cmap="viridis")
plt.axis("off")
plt.savefig(os.path.join(save_dir, "IG_Heatmap.png"), bbox_inches='tight', pad_inches=0.1)

# Overlay (Heatmap + Original)
plt.subplot(3,1,3)
plt.title("Overlay")
heatmap_ig = cv2.applyColorMap(np.uint8(255 * attr), cv2.COLORMAP_VIRIDIS)
heatmap_ig = np.float32(heatmap_ig) / 255
overlay_ig = heatmap_ig + original_np
overlay_ig = overlay_ig / np.max(overlay_ig)
plt.imshow(overlay_ig)
plt.axis("off")
plt.savefig(os.path.join(save_dir, "Overlay.png"), bbox_inches='tight', pad_inches=0.1)

plt.tight_layout()
plt.show()

print(f"Saved all images to '{save_dir}/'")


# getting the misclassfied images

In [ ]:
import os
import shutil

# --- DELETE old misclassified folder if it exists ---
if os.path.exists(save_misclassified_dir):
    shutil.rmtree(save_misclassified_dir)
os.makedirs(save_misclassified_dir, exist_ok=True)

In [ ]:
import os
import shutil
import torch
from glob import glob
from PIL import Image
import numpy as np
from ultralytics import YOLO

# --- Paths ---
image_folder = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/"
save_misclassified_dir = "misclassified"

# --- DELETE old misclassified folder if it exists ---
if os.path.exists(save_misclassified_dir):
    shutil.rmtree(save_misclassified_dir)
os.makedirs(save_misclassified_dir, exist_ok=True)

# --- Load Model ---
model_path = "/kaggle/input/trained_yolo-model/pytorch/default/1/best.pt"
model = YOLO(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# --- Class Mapping ---
class_names = model.names

# --- Thresholds ---
CONFIDENCE_THRESHOLD = 0.7   # probability threshold for "good" detection

# --- Main Logic ---
misclassified_count = 0
max_to_find = 5

image_paths = sorted(glob(os.path.join(image_folder, "*.png")))

for img_path in image_paths:
    img = Image.open(img_path).convert("RGB")
    
    results = model.predict(img, imgsz=(640, 640), conf=0.1)  # Keep low conf initially
    preds = results[0].boxes

    suspicious = False

    if preds is None or len(preds) == 0:
        # No detections at all
        suspicious = True
    else:
        pred_classes = preds.cls.cpu().numpy()
        pred_scores = preds.conf.cpu().numpy()
        pred_class_names = [class_names[int(c)].lower() for c in pred_classes]

        # Check conditions:
        if pred_scores.max() < CONFIDENCE_THRESHOLD:
            suspicious = True
        elif not any(cls in ['car', 'truck', 'van'] for cls in pred_class_names):
            suspicious = True

    if suspicious:
        filename = os.path.basename(img_path)
        save_path = os.path.join(save_misclassified_dir, filename)
        shutil.copy(img_path, save_path)
        misclassified_count += 1
        print(f"Saved suspicious image: {save_path}")

    if misclassified_count >= max_to_find:
        break

print(f"✅ Done! Saved {misclassified_count} suspicious/misclassified images in '{save_misclassified_dir}/'")
